In [ ]:
import numpy as np
import torch
from matplotlib import pyplot as plt

from src.datasets import CFMGraphDataset
from src.loaders import get_test_loader, get_train_loaders
from src.models import GATEncoder
from src.utils import predict, save

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data
train_loader, val_loader = get_train_loaders(
    batch_size=128, shuffle=False, dataset=CFMGraphDataset
)
test_loader = get_test_loader(batch_size=128, dataset=CFMGraphDataset)
is_torch_geometric = True

model = GATEncoder(
    d_features=7,
    d_edges=5,
    d_out=24,
    d_hidden_dim=300,
    num_layers=3,
    num_heads=3,
    d_linear_layers=[256],
    dropout=0.01,
    activation="ReLU",
).to(device)

load = "runs/night_test"

model.load_state_dict(torch.load(f"{load}/model.pt"))

# Test prediction
test_y_pred, _, test_probas = predict(
    model, test_loader, device, is_torch_geometric=is_torch_geometric
)
save(test_y_pred, "solution.csv")

# Score on validation set
val_y_pred, val_y_true, val_probas = predict(
    model, val_loader, device, is_torch_geometric=is_torch_geometric
)
print("Validation accuracy:", (val_y_pred == val_y_true).mean())

# Score on full training set
train_y_pred, train_y_true, train_probas = predict(
    model, train_loader, device, is_torch_geometric=is_torch_geometric
)
print("Train accuracy:", (train_y_pred == train_y_true).mean())

# Accuracies for each class
val_accuracies = []
for i in range(24):
    val_accuracies.append((val_y_pred[val_y_true == i] == i).mean())
train_accuracies = []
for i in range(24):
    train_accuracies.append((train_y_pred[train_y_true == i] == i).mean())

width = 0.2
plt.bar(np.arange(24) - width, val_accuracies, label="val", width=width)
plt.bar(np.arange(24) + width, train_accuracies, label="train", width=width)
plt.xticks(range(24))
plt.legend()
plt.show()

In [ ]:
# histogram of the argmax of the probabilities for train, val and test

plt.hist(
    np.max(train_probas, axis=1),
    bins=24,
    alpha=0.5,
    label="train",
    density=True,
)
plt.hist(
    np.max(val_probas, axis=1),
    bins=24,
    alpha=0.5,
    label="val",
    density=True,
)
plt.hist(
    np.max(test_probas, axis=1),
    bins=24,
    alpha=0.5,
    label="test",
    density=True,
)
plt.legend()
plt.show()

In [ ]:
# Class distribution
plt.hist(train_y_pred, bins=24, alpha=0.5, label="train", density=True)
plt.hist(val_y_pred, bins=24, alpha=0.5, label="val", density=True)
plt.hist(test_y_pred, bins=24, alpha=0.5, label="test", density=True)
plt.legend()
plt.show()

In [ ]:
from betacal import BetaCalibration
from sklearn.calibration import calibration_curve
from sklearn.isotonic import IsotonicRegression

K = 3
fig, ax = plt.subplots(24 // K, K, figsize=(20, 30))
bins = 10
strategy = "uniform"
# for each class
for i in range(24):
    # Without calibration
    val_prob_true, val_prob_pred = calibration_curve(
        val_y_true == i,
        val_probas[:, i],
        n_bins=bins,
        strategy=strategy,
    )
    train_prob_true, train_prob_pred = calibration_curve(
        train_y_true == i,
        train_probas[:, i],
        n_bins=bins,
        strategy=strategy,
    )
    test_prob_true, test_prob_pred = calibration_curve(
        test_y_pred == i,
        test_probas[:, i],
        n_bins=bins,
        strategy=strategy,
    )

    # With beta calibration
    calibrator = IsotonicRegression()
    calibrator.fit(train_probas[:, i].reshape(-1, 1), (train_y_true == i).astype(int))

    val_prob_pred_calibrated = calibrator.predict(val_probas[:, i].reshape(-1, 1))
    train_prob_pred_calibrated = calibrator.predict(train_probas[:, i].reshape(-1, 1))
    test_prob_pred_calibrated = calibrator.predict(test_probas[:, i].reshape(-1, 1))

    cal_val_prob_true, cal_val_prob_pred = calibration_curve(
        val_y_true == i,
        val_prob_pred_calibrated,
        n_bins=bins,
        strategy=strategy,
    )
    cal_train_prob_true, cal_train_prob_pred = calibration_curve(
        train_y_true == i,
        train_prob_pred_calibrated,
        n_bins=bins,
        strategy=strategy,
    )
    cal_test_prob_true, cal_test_prob_pred = calibration_curve(
        test_y_pred == i,
        test_prob_pred_calibrated,
        n_bins=bins,
        strategy=strategy,
    )

    # New
    ax[i // K, i % K].plot(
        cal_train_prob_pred,
        cal_train_prob_true,
        marker="o",
        label="train",
        color="blue",
    )
    ax[i // K, i % K].plot(
        cal_val_prob_pred, cal_val_prob_true, marker="o", label="val", color="orange"
    )
    ax[i // K, i % K].plot(
        cal_test_prob_pred, cal_test_prob_true, marker="o", label="test", color="green"
    )
    # Old
    ax[i // K, i % K].plot(
        train_prob_pred,
        train_prob_true,
        marker="o",
        alpha=0.5,
        color="blue",
    )
    ax[i // K, i % K].plot(
        val_prob_pred, val_prob_true, marker="o", alpha=0.5, color="orange"
    )
    ax[i // K, i % K].plot(
        test_prob_pred,
        test_prob_true,
        marker="o",
        alpha=0.5,
        color="green",
    )
    ax[i // K, i % K].plot([0, 1], [0, 1], linestyle="--", color="black")
    ax[i // K, i % K].set_title(f"Class {i+1}")
    ax[i // K, i % K].set_xlabel("Mean predicted probability")
    ax[i // K, i % K].set_ylabel("Fraction of positives")
    ax[i // K, i % K].legend()


fig.tight_layout()
plt.show()